# Requierments


In [ ]:
%pip install hazm
%pip install camel_tools
%pip install dadmatools
%pip install nltk
%pip install english-text-normalization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.2/418.2 KB 29.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for camel-kenlm: filename=camel_kenlm-2021.12.27-cp38-cp38-linux_x86_64.whl size=2968607 sha256=e966f20151fd94456509cb12bd44fff9c573ef8b28c761f3c51cf75cab217dcf
  Stored in directory: /root/.cache/pip/wheels/2c/7b/f0/837fcdb48cd99564b1163d90392f350cb933fce3bf122eadcd
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=dd307577011a89d2cd84c9144a78f7026623b676d91f4c11b14d7bb13be0d1cd
  Stored in dire

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import random as rd
import json
import numpy as np
import re
import numpy as np
import nltk
import string
from __future__ import unicode_literals
from hazm import *
nltk.download('punkt')
from nltk.tokenize import word_tokenize  
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_alef_bw
from camel_tools.utils.normalize import normalize_alef_hsb
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar
from dadmatools.models.normalizer import Normalizer

# Data Pruning
Creating light-weighted version of datasets

In [ ]:
data_set_root = "/content/drive/MyDrive/Text_enrich/datasets/"


Pruning English quotes



In [ ]:
import pandas as pd    
jsonObj = pd.read_json(path_or_buf= data_set_root +'quotes/quotes.jsonl', lines=True)

for i in range(0, len(jsonObj)):
  if 30 > len(jsonObj["quote"][i]) > 210:
    jsonObj = jsonObj.drop([i])
    
with open(data_set_root + "quotes/light_quotes.jsonl", "w") as f:
    f.write(jsonObj.to_json(orient='records', lines=True, force_ascii=False))
f.close()

Pruning Nahj-al-balagha data set

In [ ]:
columns = ["output_text" , "vector", "output_data_source","index"]
Nahj_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'nahj/Nahj Al-Balaqa.txt', 'r')
Lines = file1.readlines()
nahj = []
count = 1
i = 0
new_file = open(data_set_root + "light_nahj.txt", "w")

for line in Lines:
    splited = line.split("\t")
    i +=1
    if len(splited[1]) < 400:
      new_file.write(str(splited[0]) + "\t" + splited[1])
file1.close()
new_file.close()


Pruning Moulavi data set

In [ ]:
moulavi_qazals_path = data_set_root + "moulavi/ultra_light_divane-e-shams.txt"
moulavi_qazals_pattern = re.compile(r'\n?\s*\d+\s*?\n')

def read_divan_shams():
    with open(moulavi_qazals_path, encoding='utf-8') as f:
        text = f.read() 
        
    qazals_arr = moulavi_qazals_pattern.split(text)
    return qazals_arr[1:]


In [ ]:
columns = ["output_text" , "vector", "output_data_source","index"]
molavi_table = pd.DataFrame([], columns=columns)
molavi_data_len = len(read_divan_shams())
input_set = read_divan_shams()
new_file = open(data_set_root + "moulavi/light_divane-e-shams.txt", "w")
for i in range(0, molavi_data_len):
  if len(input_set[i]) < 600:
    input = input_set[i]
    new_file. write(str(i) + '\n'+ input + '\n\n')
new_file.close()

# Prepration Normalized Data from pruned Data


In [ ]:
pd.DataFrame(input).to_csv("/content/input_text.csv")

In [ ]:
input

<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x7f26c93cf2b0>>

In [ ]:
columns = ["input_text" , "vector"]
input_table = pd.DataFrame([], columns=columns)
input = pd.DataFrame([], columns=["input_id","input_text"])

normalizer = Normalizer()
data = data.sample(frac = 1).reset_index()
for index in range(len(data)):
  sample = data.iloc[index]["input_text"]
  input_table = input_table.append({'input_text':sample,'vector':[embedder.encode(normalizer.normalize(sample))]}, ignore_index = True)
  input = input.append({'input_id':index,'input_text':sample}, ignore_index = True)
  

In [ ]:
def normalize_sentence(tokenized_sents, minimum_length=0, stopword_removal=False, stopwords_domain=[], lower_case=False, punctuation_removal=True):
    '''
    normalization function
    '''
    sentence = tokenized_sents
    
    if stopword_removal:
        # Remove stopwords in English and also the given domain stopwords
        stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]
        normalized_sents=[word for word in sentence if (word.lower() not in stopwords_domain + stopwords)]

    if punctuation_removal:
        # Remove punctuations
        normalized_sents=[word for word in sentence if word not in string.punctuation]

    if lower_case:
        # Convert everything to lowercase and filter based on a min length
        normalized_sents=[word.lower() for word in sentence if len(word)>minimum_length]

    elif minimum_length>1:
        normalized_sents= [word for word in sentence if len(word)>minimum_length]        
        
    return normalized_sents

In [ ]:
output_english = pd.read_csv("/content/light_quotes.csv")

In [ ]:
columns = ["output_text" , "vector", "output_data_source","index"]
English_qoute_table_legacy = pd.DataFrame([], columns=columns)
columns_2 = ["output_id" , "output_text"]
quotes_normalized_2 = pd.DataFrame([], columns=columns_2)


f = open(data_set_root +'quotes/quotes.jsonl')
quotes = [json.loads(jline) for jline in f.read().splitlines()]
n = 0
for i in range(0, len(quotes)):
  if 20 < len(quotes[i]["quote"]) < 210:
    quotes_normalized_2 = quotes_normalized_2.append({'output_id': n + 3223, 'output_text': quotes[i]["quote"][1:-2]}, ignore_index = True)
    English_qoute_table_legacy = English_qoute_table_legacy.append({'output_text':quotes[i]["quote"][1:-2],'vector':laser.embed_sentences(quotes[i]["quote"][1:-2], 'en'), 'output_data_source':'نقل قول از نویسندگان انگلیسی', 'index':n}, ignore_index = True)
    n += 1
f.close()
pd.DataFrame(quotes_normalized_2).to_csv(data_set_root+ "quotes/light_quotes.csv")

In [ ]:
columns = ["output_id","output_text"]
quotes_normalized = pd.DataFrame([], columns=columns)
for i in range(len(output_english)):
  row = output_english.iloc[i]["output_text"]
  row = row.replace("â€™"," ")
  row = row.replace("â€¦"," ")
  row = row.replace("â€²"," ")
  row = row.replace("â€�â€œ"," ")
  row = row.replace("â€”"," ")
  quotes_normalized = quotes_normalized.append({'output_id':i+3208 , 'output_text':row}, ignore_index = True)





In [ ]:
output_english.iloc[12]["output_text"] = output_english.iloc[12]["output_text"].replace("â€¦"," ").replace("â€™"," ")

/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [ ]:
quotes_normalized

,output_id,output_text
0,3208,Be yourself; everyone else is already taken
1,3209,"I'm selfish, impatient and a little insecure. ..."
2,3210,Two things are infinite: the universe and huma...
3,3211,"So many books, so little time"
4,3212,A room without books is like a body without a ...
...,...,...
2036,5244,"Stepping onto a brand-new path is difficult, b..."
2037,5245,Morality is simply the attitude we adopt towar...
2038,5246,"In life, finding a voice is speaking and livin..."
2039,5247,"Winter is the time for comfort, for good food ..."


In [ ]:
pd.DataFrame(output_english).to_csv("/content/quotes_normalized.csv")

In [ ]:


columns = ["output_id","output_text"]
quotes_normalized = pd.DataFrame([], columns=columns)
f = open('/content/light_quotes.csv')
# quotes = json.load(f)
quotes = [json.loads(jline) for jline in f.read().splitlines()]
# print(quotes) 
j = 0
for i in range(len(quotes)):
    if(len(quotes[i]["quote"][1:-2]) < 300 and len(quotes[i]["quote"][1:-2]) >30 and not("Ø" in quotes[i]["quote"]) and not("�" in quotes[i]["quote"])):
      
      quotes[i]["quote"] = quotes[i]["quote"].replace("â€™"," ")
      quotes[i]["quote"] = quotes[i]["quote"].replace("â€¦"," ")
      quotes[i]["quote"] = quotes[i]["quote"].replace("â€ ”"," ")
      quotes_normalized = quotes_normalized.append({'output_id':i + 3208 -j , 'output_text':' '.join(normalize_sentence(word_tokenize(quotes[i]["quote"][1:-2])))}, ignore_index = True)
    else:
      j += 1
f.close()


In [ ]:
quotes_normalized

,output_id,output_text
0,3208,Be yourself; everyone else is already taken
1,3209,"I'm selfish, impatient and a little insecure. ..."
2,3210,Two things are infinite: the universe and huma...
3,3211,"So many books, so little time"
4,3212,A room without books is like a body without a ...
...,...,...
2036,5244,"Stepping onto a brand-new path is difficult, b..."
2037,5245,Morality is simply the attitude we adopt towar...
2038,5246,"In life, finding a voice is speaking and livin..."
2039,5247,"Winter is the time for comfort, for good food ..."


In [ ]:
pd.DataFrame(quotes_normalized).to_csv("/content/quotes_normalized.csv")

In [ ]:
output_table1

,output_text,vector,output_data_source,index
0,Be yourself; everyone else is already taken,"[[0.01367188, -0.068347186, -0.019004673, 0.03...",عبارت های انگلیسی,1
1,"I'm selfish, impatient and a little insecure. ...","[[-0.04042464, -0.0022568868, -0.05183047, -0....",عبارت های انگلیسی,2
2,Two things are infinite: the universe and huma...,"[[-0.053514086, -0.0064785215, -0.062029704, -...",عبارت های انگلیسی,3
3,"So many books, so little time","[[-0.06357765, -0.059847917, -0.03633031, -0.0...",عبارت های انگلیسی,4
4,A room without books is like a body without a ...,"[[-0.039728824, 0.036270317, -0.06372754, -0.0...",عبارت های انگلیسی,5
...,...,...,...,...
2501,Morality is simply the attitude we adopt towar...,"[[0.013751664, 0.04087357, -0.034762938, -0.02...",عبارت های انگلیسی,2502
2502,Don't aim at success. The more you aim at it a...,"[[-0.02788582, -0.025344137, 0.00016675645, -0...",عبارت های انگلیسی,2503
2503,"In life, finding a voice is speaking and livin...","[[-0.07175418, -0.058423884, -0.019333908, -0....",عبارت های انگلیسی,2504
2504,"Winter is the time for comfort, for good food ...","[[-0.017963018, -0.049638994, -0.051915742, -0...",عبارت های انگلیسی,2505


In [ ]:


def normalize_arabic(sentence):

    # Normalize alef variants to 'ا'
    sent_norm = normalize_unicode(sentence)
    
    sent_norm = normalize_alef_bw(sent_norm)
    # Normalize alef variants to 'ا'
    sent_norm = normalize_alef_ar(sentence)

    # Normalize alef maksura 'ى' to yeh 'ي'
    sent_norm = normalize_alef_maksura_ar(sent_norm)

    # Normalize teh marbuta 'ة' to heh 'ه'
    sent_norm = normalize_teh_marbuta_ar(sent_norm)
    return dediac_ar(sent_norm)

In [ ]:
columns = ["output_id","output_text","hekmat_number"]
nahj_normalized = pd.DataFrame([], columns=columns)
file1 = open('/content/light_nahj.txt', 'r')
Lines = file1.readlines()
nahj = []  

hekmat = r".*2##[0-9]+.*"
hekmat_reg = f"{hekmat}" 
i = 5249
hek = []
for line in Lines:
  
  if(re.match(hekmat_reg, line)):
    # print(count)
    # print(line)
    
    
    splited = line.split("\t") 
    input = splited[1].replace("\n"," ")
    print(splited[1])
    nahj_normalized = nahj_normalized.append({'output_id': i , 'output_text':normalize_arabic(input),'hekmat_number':(splited[0].split("##"))[1] }, ignore_index = True)
    i+= 1
  else:
    hek.append(line.strip())
file1.close()


In [ ]:
nahj_normalized

,output_id,output_text,hekmat_number
0,5249,قال ع کن فی الفتنه کابن اللبون لا ظهر فیرکب و ...,1
1,5250,و قال ع ازری بنفسه من استشعر الطمع و رضی‌ بالذ...,2
2,5251,و قال ع البخل عار و الجبن منقصه و الفقر یخرس ا...,3
3,5252,و قال ع العجز افه و الصبر شجاعه و الزهد ثروه و...,4
4,5253,و قال ع العلم وراثه کریمه و الاداب حلل مجدده و...,5
...,...,...,...
429,5678,و قال ع لزیاد ابن ابیه و قد استخلفه لعبد الله ...,476
430,5679,و قال ع اشد الذنوب ما استخف بها صاحبه,477
431,5680,و قال ع ما اخذ الله علی اهل الجهل ان یتعلموا ح...,478
432,5681,و قال ع شر الاخوان من تکلف له,479


In [ ]:
pd.DataFrame(nahj_normalized).to_csv("/content/nahj_normalized.csv")

In [ ]:


dadma_normalizer = Normalizer(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with="<NUM>",
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None,
)

columns = ["output_id","output_text"]
poem_normalized = pd.DataFrame([], columns=columns)

file1 = open('/content/yek-beiti.txt', 'r')
Lines = file1.readlines()
i = 2873
for line in Lines:
    line = line.replace("/"," ")
    # print(line)
    
    poem_normalized = poem_normalized.append({'output_id': i,'output_text':dadma_normalizer.normalize(line)}, ignore_index = True)
    i+=1
file1.close()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
poem_normalized

,output_id,output_text
0,2873,در سخن گفتن خطای جاهلان پیدا شود تیر کج چون از...
1,2874,ترا دیدم و یوسف را شنیدم شنیدن کی بود مانند دیدن
2,2875,دل ز پر گفتن بمیرد در بدن گر چه گفتارش بود در عدن
3,2876,امشب تو را به خوبی نسبت به ماه کردم تو خوب تر ...
4,2877,به همان قدر که چشم تو پر از زیباییست بی تو دنی...
...,...,...
330,3203,در این عالم از هرچه بایستنیست نکوتر ز فرزند شا...
331,3204,در جوانی پاک بودن شیوه پیغمبریست ورنه هر گبری ...
332,3205,درست نیست دو رنگی میان ظاهر و باطن بگو شکسته ن...
333,3206,در غربت مرگ بیم تنهایی نیست یاران عزیز آن طرف ...


In [ ]:
pd.DataFrame(poem_normalized).to_csv("/content/poem_normalized.csv")

In [ ]:


dadma_normalizer = Normalizer(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with="<NUM>",
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None,
)


columns = ["output_id","output_text","qazal_number"]
hafez_normalized = pd.DataFrame([], columns=columns)
file1 = open('/content/hafez-qazals.txt', 'r')
Lines = file1.readlines()
i = 0
flag = False
qazals = []
for line in Lines:
    # line = line.replace("/","")
    # print(line)
    qazal = r".*غزل.*"
    qazal_reg = f"{qazal}"    
    if(re.match(qazal_reg, line) and len(line)<20):
      
      # print(line)
      input =  ' '.join(qazals)
      # print("_"*100)
      if(len(input)>5):
        hafez_normalized = hafez_normalized.append({'output_id':i-1,'output_text':dadma_normalizer.normalize(input),'qazal_number':prev_line}, ignore_index = True)
      qazals = []
      i += 1
      line = line.replace("\n","")
      prev_line = line
    else:
      if(len(line)>5): 
        qazals.append(line.strip())
input =  ' '.join(qazals)
if(len(input)>5):
    hafez_normalized = hafez_normalized.append({'output_id':i-1,'output_text':dadma_normalizer.normalize(input),'qazal_number':prev_line}, ignore_index = True)  
file1.close()


In [ ]:
hafez_normalized

,output_id,output_text,qazal_number
0,0,الا یا ایها الساقی ادر کاسا و ناولها که عشق آس...,﻿غزل ۱
1,1,صلاح کار کجا و من خراب کجا ببین تفاوت ره کز کج...,غزل ۲
2,2,اگر آن ترک شیرازی به دست آرد دل ما را به خال ه...,غزل ۳
3,3,صبا به لطف بگو آن غزال رعنا را که سر به کوه و ...,غزل ۴
4,4,دل می‌رود ز دستم صاحب دلان خدا را دردا که راز ...,غزل ۵
...,...,...,...
490,490,به چشم کرده‌ام ابروی ماه سیمایی خیال سبزخطی نق...,غزل ۴۹۱
491,491,سلامی چو بوی خوش آشنایی بدان مردم دیده روشنایی...,غزل ۴۹۲
492,492,ای پادشه خوبان داد از غم تنهایی دل بی تو به جا...,غزل ۴۹۳
493,493,ای دل گر از آن چاه زنخدان به درآیی هر جا که رو...,غزل ۴۹۴


In [ ]:
pd.DataFrame(hafez_normalized).to_csv("/content/hafez_normalized.csv")

In [ ]:


dadma_normalizer = Normalizer(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with="<NUM>",
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None,
)

columns = ["output_id","output_text","qazal_number"]
shams_normalized = pd.DataFrame([], columns=columns)
file1 = open('/content/ultra_light_divane-e-shams.txt', 'r')
Lines = file1.readlines()
spilt = r".*[0-9]+.*"
spilt_reg = f"{spilt}" 
i = 495
flag = False
qazals = []
for line in Lines:
  if(re.match(spilt_reg, line) and len(line)<20):
    
    # print(line)
    input =  ' '.join(qazals)
    # print(input)
    # print("_"*100)
    if(len(input)>5):

      shams_normalized = shams_normalized.append({'output_id':i , 'output_text':dadma_normalizer.normalize(input),'qazal_number':int(prev_line)+1}, ignore_index = True)
      i += 1
    qazals = []
    line = line.replace("\n","")
    prev_line = line
  else:
    if(len(line)>5): 
      qazals.append(line.strip())

input =  ' '.join(qazals)
if(len(input)>5):
  shams_normalized = shams_normalized.append({'output_id':i , 'output_text':dadma_normalizer.normalize(input),'qazal_number':int(prev_line)+1}, ignore_index = True)

file1.close() 


In [ ]:
shams_normalized

,output_id,output_text,qazal_number
0,495,ای رستخیز ناگهان، وی رحمت بی منتها ای آتشی افر...,1
1,496,ای یوسف خوش نام ما خوش می روی بر بام ما ای درش...,4
2,497,بگریز ای میر اجل از ننگ ما از ننگ ما زیرا نمی ...,6
3,498,جز وی چه باشد کز اجل اندررباید کل ما صد جان بر...,8
4,499,من از کجا پند از کجا باده بگردان ساقیا آن جام ...,9
...,...,...,...
2373,2868,یا ملک المحشر، ترحم لا ترتشی کل سقیط ردی ترحمه...,3225
2374,2869,قلت له مصیحا یا ملک المشرق اقسم بالخالق مثلک ل...,3226
2375,2870,یا ساقی الراح خذ و امرلائ به طاسی فلست املک صب...,3227
2376,2871,ایا ملتقی العیش کم تبعدی و یا فرقه الحسب کم تع...,3228


In [ ]:
pd.DataFrame(shams_normalized).to_csv("/content/shams_normalized.csv")